In [1]:
#%matplotlib inline
import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
from siamese.dataset import Dataset
from siamese.dataset import DataLoader
from siamese.model import *
from extensies import metrics as mt
from extensies import preprocessing as ps
from matplotlib import gridspec

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
import pandas as pd 
import ast

columns = ['date','name','modality','size','model','num_of_data','iterations','augmentation',
           'normalization','acc','acc_w','acc_p','auc','auc_w','auc_p','komentar']

df = pd.read_csv('records.csv',index_col=0)

In [3]:
df.tail(10)

,date,name,modality,size,model,num_of_data,iterations,augmentation,normalization,acc,acc_w,acc_p,auc,auc_w,auc_p,komentar
44,2019-04-06,adc_20x20x1_32844,['adc/t/20x20x1'],20x20x1,siamese,296,1000,False,ScaleNormalization,0.764045,0.348315,0.707865,0.543233,0.566541,0.507519,NaN
45,2019-04-06,adc_20x20x1_98058,['adc/t/20x20x1'],20x20x1,siamese,296,1000,False,ScaleNormalization,0.752809,0.325843,0.752809,0.516917,0.513910,0.631955,NaN
46,2019-04-06,adc_t2tsetra_ktrans_20x20x1_12759,"['adc/t/20x20x1', 't2tsetra/t/20x20x1', 'ktran...",20x20x1,xmass,296,1000,False,ScaleNormalization,0.786517,0.325843,0.775281,0.557519,0.513910,0.684586,NaN
47,2019-04-06,adc_t2tsetra_ktrans_20x20x1_37047,"['adc/t/20x20x1', 't2tsetra/t/20x20x1', 'ktran...",20x20x1,xmass,296,1500,False,ScaleNormalization,0.764045,0.393258,0.719101,0.543233,0.537594,0.610526,NaN
48,2019-04-06,adc_t2tsetra_20x20x1_10921,"['adc/t/20x20x1', 't2tsetra/t/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.797753,0.325843,0.752809,0.545489,0.571429,0.574436,NaN
49,2019-04-06,adc_t2tsetra_20x20x1_19040,"['adc/t/20x20x1', 't2tsetra/t/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.820225,0.325843,0.730337,0.617293,0.533083,0.656015,NaN
50,2019-04-06,adc_t2tsetra_20x20x1_26758,"['adc/s/20x20x1', 't2tsetra/s/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.775281,0.258427,0.696629,0.492857,0.451880,0.500376,NaN
51,2019-04-06,adc_t2tsetra_20x20x1_73093,"['adc/s/20x20x1', 't2tsetra/s/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.764045,0.280899,0.719101,0.504887,0.485338,0.572180,NaN
52,2019-04-06,adc_t2tsetra_20x20x1_69066,"['adc/a/20x20x1', 't2tsetra/a/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.741573,0.325843,0.662921,0.490602,0.513910,0.478947,NaN
53,2019-04-06,adc_t2tsetra_20x20x1_15279,"['adc/a/20x20x1', 't2tsetra/a/20x20x1']",20x20x1,xmass,296,1000,False,ScaleNormalization,0.786517,0.415730,0.752809,0.576692,0.571053,0.651128,NaN


In [4]:
idxs = [49,51,53]
datasets = {}
for idx in idxs:
    model_name = df.iloc[idx]['name']
    modalities = df.iloc[idx]['modality']
    modalities = ast.literal_eval(modalities)
    combined = len(modalities) > 1
    print('Model name: ',model_name)
    print('Model type: ',df.iloc[idx]['model'])

    loader = DataLoader('../../data/',modalities)
    loader.load_data()

    if combined:
        loader = DataLoader('../../data/',modalities)
        loader.load_data()
        X,y = loader.combine_channels(modalities)
        X_train, X_test, y_train, y_test = loader.get_train_test('combined')

    else:
        loader = DataLoader('../../data/',modalities)
        loader.load_data()
        X,y = loader.get_data(modalities[0])
        X_train, X_test, y_train, y_test = loader.get_train_test(modalities[0])

    print("Data shape: ",X_train[0].shape)

    datasets[idx] = Dataset()
    datasets[idx].images_train = X_train
    datasets[idx].images_test = X_test
    datasets[idx].labels_train = y_train
    datasets[idx].labels_test = y_test


Model name:  adc_t2tsetra_20x20x1_19040
Model type:  xmass
(20, 20)
(20, 20)
(20, 20)
(20, 20)
Data shape:  (20, 20, 2)
Model name:  adc_t2tsetra_20x20x1_73093
Model type:  xmass
(20, 20)
(20, 20)
(20, 20)
(20, 20)
Data shape:  (20, 20, 2)
Model name:  adc_t2tsetra_20x20x1_15279
Model type:  xmass
(20, 20)
(20, 20)
(20, 20)
(20, 20)
Data shape:  (20, 20, 2)


In [5]:
img_placeholder = tf.placeholder(tf.float32, [None] + list(datasets[idxs[0]].images_train.shape[1:]), name='img')
net = xmas_model(img_placeholder, reuse=False)

In [6]:
train_feat = {}
search_feat = {}
for idx in idxs:
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state("model")
        saver.restore(sess, "model/" + df.iloc[idx]['name'] + ".ckpt")
        train_feat[idx] = sess.run(net, feed_dict={img_placeholder:datasets[idx].images_train})
        search_feat[idx] = sess.run(net, feed_dict={img_placeholder:datasets[idx].images_test})
    

INFO:tensorflow:Restoring parameters from model/adc_t2tsetra_20x20x1_19040.ckpt
INFO:tensorflow:Restoring parameters from model/adc_t2tsetra_20x20x1_73093.ckpt
INFO:tensorflow:Restoring parameters from model/adc_t2tsetra_20x20x1_15279.ckpt


In [26]:
y_pred = {}
y_pred_t = {}
y_pred_w = {}
for idx in idxs:
    y_pred[idx] = []
    y_pred_t[idx] = []
    y_pred_w[idx] = []
for idx in idxs:
    for feat in search_feat[idx]:
        #calculate the cosine similarity and sort
        y_pred[idx].append(mt.siamese_predict(train_feat[idx],feat,datasets[idx]))
        y_pred_t[idx].append(mt.treshold_predict(train_feat[idx],feat,datasets[idx],0.5,10,distance= 'euclidean'))
        y_pred_w[idx].append(mt.weighted_predict(train_feat[idx],feat,datasets[idx],0.5,10))

In [27]:
accuracy_score(datasets[49].labels_test,y_pred_t[49])

0.7865168539325843

In [50]:
metrics.roc_auc_score(datasets[49].labels_test,pred)

0.5815789473684211

In [44]:
pred1 = zip(y_pred_t[49], y_pred[49])
pred2 = zip(y_pred_t[51], y_pred[51])
pred3 = zip(y_pred_t[53], y_pred[53])

In [45]:
pred1 = [0 if 0  in x else 1 for x in pred1]
pred2 = [0 if 0  in x else 1 for x in pred2]
pred3 = [0 if 0  in x else 1 for x in pred3]
# metrics.roc_auc_score(datasets[49].labels_test,pred)

In [48]:
pred =  [1 if 1 in x else 0 for x in zip(pred1,pred2,pred3)]

In [49]:
pred

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0]